In [1]:
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm # to create loadbard in for loop
import os.path

# What will be achieved
This file will download the comments of the earning transcripts downloaded in the article folder.
The comments will be saved in the comments folder.
Note that normal get request, flagged the bot detection.
The code can be adapted to check, if we already have the comments so that it do not fetch them again. However, this would mean that new comments will not be downloaded.

In [4]:
id_earning_calls = [name[:-5] for name in os.listdir("data/article/")]
executable_path = "./chromedriver_linux"
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path= executable_path,options=option)
for id_earning_call in tqdm(id_earning_calls):
    with open("data/comment/"+ id_earning_call+".json", "w") as file:
        url = "https://seekingalpha.com/api/v3/articles/"\
            +id_earning_call\
            +"/comment_maps?include=user&sort=-top_parent_id"
        driver.get(url)
        api_json_response = driver.find_element_by_tag_name("body").text
        file.write(api_json_response)

/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.
  0%|                                                                                                                                                                                                                                                   | 0/57 [00:00<?, ?it/s]/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  # This is added back by InteractiveShellApp.init_path()
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:20<00:00,  2.75i

In [5]:
#sanity check of last reponse no captcha/bot detection
print(api_json_response)

{"data":[],"included":[],"meta":{"page":{"size":0,"total":null}},"links":{"next":null,"self":"/articles/4505103/comment_maps?include=user&sort=-top_parent_id"}}
